# Content2Form

In questo esercizio l'obiettivo è trovare il concetto a cui fanno riferimento alcune definizioni. In particolare dovremo trovare 4 concetti che si distinguono su due dimensioni: Concreto/Astratto e Specifico/Generico.

Il metodo di risoluzione per singolo concetto prevede l'individuazione dei genus tra le parole che ricorrono maggiormente nelle definizioni afferenti a quel concetto. Partendo dai synset relativi ai genus si cercano i synset che potrebbero corrispondere ai concetti con una visita esaustiva degli iponimi. Ad ognuno dei synset così trovati viene assegnato un punteggio calcolato sulla base della somiglianza tra la definizione del synset stesso e le parole più frequenti nelle definizioni del concetto. I synset che massimizzano questo punteggio dovrebbero essere molto probabilmente i synset associati al concetto originale.

## Import

In [27]:
import csv
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

from gensim.test.utils import simple_preprocess
from collections import Counter
from funcy import merge_with

import warnings
warnings.filterwarnings('ignore')

## Preprocessing

In [28]:
def preprocess_string(sent):
    sent = simple_preprocess(sent)
    res = [word for word in sent if word not in stopwords.words("english")]
    return res

## Recupero delle definizioni

Il primo step prevede di recuperare le definizioni dal file in cui sono memorizzate.

In [29]:
def get_definitions(path):
    res = []
    with open(path, 'r') as file:
        defs = csv.reader(file)
        _ = next(defs)
        for row in defs:
            res.append(row[1:])
    return res

get_definitions('resources/definizioni.csv')

[['Range of concepts human beings feel in certain situations',
  'Something you can feel',
  'Something that an animal can feel',
  'something you think that makes you feel good or bad',
  'Human sensation arising from the form of feelings',
  'State of mind that a living being can percieve',
  'Feeling that a human or an animal can express towards others',
  'A sentiment that a living entity can feel and express throw words and their body.',
  'what you feel in a certain moment',
  "feeling experienced by sentient beings and cause by events happening in the outside world as well as in the being's mind",
  '',
  '',
  'sentimental reation to an action',
  'something you can feel',
  'what a human being is feeling eg happyness sadness love',
  'general concept of feeling',
  'A feeling deriving from human life',
  'an human sensation ',
  'An animal feeling',
  'Emotion is a psychic state',
  'something that a human being can feel',
  'Human sensation that can affect someones mood in ei

## Ricerca dei genus

A partire dalle definizioni, estraiamo le parole più frequenti, dalle quali selezioneremo i genus.

In [30]:
def find_most_frequent_words(definitions, max_num):
    tmp = dict()
    for definition in definitions:
        counter = Counter(preprocess_string(definition))
        tmp = merge_with(sum, tmp, counter)
    sorted_counter = dict(sorted(tmp.items(), key=lambda x: x[1], reverse=True))
    return list(sorted_counter.keys())[:max_num]

## Ricerca dei synset

A partire dai genus raccogliamo tutti i synset che potrebbero essere uno dei concetti originali. In particolare per ogni genus consideriamo alcuni synset ad esso associati. Per ognuno di questi synset visitiamo per intero il sottoalbero di cui è radice.

In [31]:
def retrieve_synsets(genuses, max_num):
    res = set()
    res.update(get_synsets_for_genus(genuses, max_num))
    res.update(retrieve_hyponyms(res))
    return list(res)

def get_synsets_for_genus(genus, max_num):
    res = []
    for word in genus:
        res.extend(wn.synsets(word)[:max_num])
    return list(filter(lambda x: x is not None, res))

def retrieve_hyponyms(synset_list):
    res = []
    hypo = lambda s: s.hyponyms()
    for syns in synset_list:
        res.extend(syns.closure(hypo))
    return res

## Calcolo del best matching synset

Per ognuno dei synset calcoliamo un punteggio di sovrapposizione tra la definizione del synset stesso e le parole più frequenti nelle definizioni date da noi. In base a questo punteggio troviamo i synset con maggiore sovrapposizione, e che quindi saranno con maggiore probabilità i synset a cui afferisce il concetto originale.

In [32]:
def rank_synsets(synset_list, most_freq_words):
    res = []
    for syns in synset_list:
        sovr = compute_sovrapposition(syns.definition(), most_freq_words)
        if sovr != 0:
            res.append((syns, sovr))
    return list(sorted(res, key=lambda x: x[1], reverse=True))

def compute_sovrapposition(definition, most_freq_words):
    definition = preprocess_string(definition)
    return len(set(definition) & set(most_freq_words))

def filter_best_synsets(best_matching_synsets, limit):
    max_value = best_matching_synsets[0][1]
    best_matching_synsets = [x for x in best_matching_synsets if x[1] == max_value]
    return best_matching_synsets[:limit]

## Esecuzione

Nell'esecuzione del programma sono state fatte alcune scelte:
- il numero di parole più frequenti che consideriamo è 10
- i genus sono selezionati come le tre parole più frequenti nelle definizioni di un concetto
- cercando i synset a partire da un genus, cominciamo la ricerca dal primo synset restituito da wordnet per quel genus e non ne consideriamo altri
- il numero massimo di synset diversi (tutti con lo stesso punteggio) per concetto che viene restituito in output è 5.

Quest'ultima scelta, sebbene controintuitiva, si può facilmente ricondurre a un solo synset pescandone uno a caso, oppure implementando metodi di selezione più raffinati.

In [33]:
concepts = ['emotion', 'person', 'revenge', 'brick']
definitions = get_definitions('resources/definizioni.csv')

for concept in range(len(concepts)):
    most_freq_words = find_most_frequent_words(definitions[concept], 10)
    genuses = most_freq_words[:3]
    synsets = retrieve_synsets(genuses, 1)
    ranked_synsets = rank_synsets(synsets, most_freq_words)
    results = filter_best_synsets(ranked_synsets, 5)

    print(f'Original concept: "{concepts[concept]}". Genus: {genuses}')
    for syns in results:
        print(f'\t{syns[0].name()} - score: {syns[1]}')
    print()

Original concept: "emotion". Genus: ['feeling', 'human', 'feel']
	hesitance.n.01 - score: 2
	intimidation.n.03 - score: 2
	eagerness.n.01 - score: 2
	disapproval.n.01 - score: 2
	boredom.n.01 - score: 2

Original concept: "person". Genus: ['human', 'person', 'living']
	secretary_of_health_and_human_services.n.01 - score: 2
	tenderfoot.n.01 - score: 2
	agricultural_laborer.n.01 - score: 2
	man_jack.n.01 - score: 2
	plaster_saint.n.01 - score: 2

Original concept: "revenge". Genus: ['someone', 'anger', 'feeling']
	exploiter.n.01 - score: 3
	displeasure.n.01 - score: 3
	disapproval.n.01 - score: 3
	respecter.n.01 - score: 3
	mourner.n.01 - score: 3

Original concept: "brick". Genus: ['used', 'object', 'material']
	building_block.n.02 - score: 4
	brick.n.01 - score: 4
	cement.n.02 - score: 4



Possiamo vedere come il programma non abbia avuto particolarmente successo con la sola eccezione del concetto "brick". Esso è il concetto Specifico e Concreto, che già nell'esercizio Defs avevamo visto distinguersi particolarmente dagli altri per similarità tra le definizioni.